In [4]:
import math
import numpy as np
import os
import sys
from ecell4 import *
rng = GSLRandomNumberGenerator()
rng.seed(0)

D = 5.
ratio = 10
ka1, kd1, kcat1 = 0.04483455086786913, 1.35, 1.5
ka2, kd2, kcat2 = 0.09299017957780264, 1.73, 15.0
trel = 1e-6
print 'trel',trel
k7 = 700000.#math.log(2)/trel
print 1./700000
r0=1.0208582
radius = 0.005
NKT = 50
dur = 1#200#1000


@species_attributes
def attrgen(radius, D):
    K | {"radius": radius, "D": D}
    Kp | {"radius": radius, "D": D}
    Kpp | {"radius": radius, "D": D}
    KK | {"radius": radius, "D": D}
    PP | {"radius": radius, "D": D}
    K_KK | {"radius": radius, "D": D}
    Kp_KK | {"radius": radius, "D": D}
    Kpp_PP | {"radius": radius, "D": D}
    Kp_PP | {"radius": radius, "D": D}

@reaction_rules
def rulegen(kon1, koff1, kcat1, kon2, koff2, kcat2,k7):
    (KK + K == K_KK | (kon1, koff1)
        > Kp + KKa | kcat1)
    (KK + Kp == Kp_KK | (kon2, koff2)
        > Kpp + KKa | kcat2)
    (KKa > KK | k7)
    (Kpp + PP == Kpp_PP | (kon1, koff1)
        > Kp + PPa | kcat1)     
    (Kp + PP == Kp_PP | (kon2, koff2)
        > K + PPa | kcat2)
    (PPa > PP | k7)

def runs(ratio,m):
    w = spatiocyte.SpatiocyteWorld(Real3(1, 1, 1),radius,rng)
    w.bind_to(m)
    w.add_molecules(Species("K"), NKT)
    npp = int(25./(ratio+1))
    nkk = 25-npp
    w.add_molecules(Species("KK"), nkk)
    w.add_molecules(Species("KKa"), 10)
    w.add_molecules(Species("PP"), npp)
    sim = spatiocyte.SpatiocyteSimulator(m, w)
    obs = FixedIntervalNumberObserver(0.1, ["KK","Kpp","PP"])
    sim.run(dur, obs)
    data = np.array(obs.data()).T
    NKK = data[1]
    NKpp = data[2]
    NP = data[3]
    tt = data[0]
    return tt,NKK,NKpp,NP

m = NetworkModel()
for i, sp in enumerate(attrgen(str(radius), str(D))):
    print(i, sp.serial(), sp.get_attribute("radius"), sp.get_attribute("D"))
    m.add_species_attribute(sp)
rul = rulegen(ka1, kd1, kcat1, ka2, kd2, kcat2,k7)
for i, rr in enumerate(rul):
    reactants, products, k = rr.reactants(), rr.products(), rr.k()
    print(i, rr.as_string())
    m.add_reaction_rule(rr)
tt,NKK,NKpp,NP = runs(ratio,m)
meanN = np.mean(NKpp[int(dur*0.5):])/NKT




trel 1e-06
1.42857142857e-06
(0, u'K', u'0.005', u'5.0')
(1, u'Kp', u'0.005', u'5.0')
(2, u'Kpp', u'0.005', u'5.0')
(3, u'KK', u'0.005', u'5.0')
(4, u'PP', u'0.005', u'5.0')
(5, u'K_KK', u'0.005', u'5.0')
(6, u'Kp_KK', u'0.005', u'5.0')
(7, u'Kpp_PP', u'0.005', u'5.0')
(8, u'Kp_PP', u'0.005', u'5.0')
(0, u'KK+K>K_KK|0.0448346')
(1, u'K_KK>KK+K|1.35')
(2, u'K_KK>Kp+KKa|1.5')
(3, u'KK+Kp>Kp_KK|0.0929902')
(4, u'Kp_KK>KK+Kp|1.73')
(5, u'Kp_KK>Kpp+KKa|15')
(6, u'KKa>KK|700000')
(7, u'Kpp+PP>Kpp_PP|0.0448346')
(8, u'Kpp_PP>Kpp+PP|1.35')
(9, u'Kpp_PP>Kp+PPa|1.5')
(10, u'Kp+PP>Kp_PP|0.0929902')
(11, u'Kp_PP>Kp+PP|1.73')
(12, u'Kp_PP>K+PPa|15')
(13, u'PPa>PP|700000')


In [5]:
w = spatiocyte.SpatiocyteWorld(Real3(1, 1, 1),radius,rng)
sim = spatiocyte.SpatiocyteSimulator(m, w)
obs = FixedIntervalNumberObserver(1.0, ["KK","Kpp","PP"])
print [obs]

In [2]:
k=1000000
import math
import numpy as np
import os
import sys
from ecell4 import *
rng = GSLRandomNumberGenerator()
D=1

def create_model(k, D, radius=0.005):
    with species_attributes():
        A | B | {'D': str(D), 'radius': str(radius)}
    with reaction_rules():
        A  > B | k # uma^2/s
    return get_model()

def singlerun(k, D, voxel_radius, L, N,run):
    radius = voxel_radius
    NB = N
    rng.seed(run)
    m = create_model(k, D, radius)
    w = spatiocyte.SpatiocyteWorld(ones() * L, voxel_radius,rng)
    w.bind_to(m)
    w.add_molecules(Species('A'), 10)
    w.add_molecules(Species('B'), 0)
    sim = spatiocyte.SpatiocyteSimulator(w)
    print sim.next_time()
    obs = FixedIntervalNumberObserver(1,['A'])
    while len(obs.data()) == 0 or obs.data()[-1][1] > N * 0.2:
        sim.run(10, [obs])
    return obs

obs =singlerun(k,D,0.005,1,10,1)

9.68838725027e-09
